### Imports and helper functions

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from cmdstanpy import CmdStanModel
import cmdstanpy
import numpy as np
from scipy.stats import invgamma
from scipy.stats import norm
from scipy.special import softmax
from gptools.stan import get_include

import pandas as pd
from glob import glob
from nteprsm import utils 
from cmdstanpy import stanfit
from settings import ROOT_DIR
import plotly.express as px
import utils as notebook_utils
# use customize plotly template
notebook_utils.set_custom_template()

import pickle

import seaborn as sns


In [ ]:
## Helper functions
def rbf_kernel_2D(x1, x2, length_scale=1.0, variance=1.0):
    # Squared Euclidean distance
    sqdist = np.sum((x1 - x2)**2)
    return variance * np.exp(-0.5 * sqdist / length_scale**2)

def generate_plot_effect_gaussian_process(grid_width, grid_height, length_scale=1.0, variance=1.0, mean=0.0):
    # Create the grid
    x = np.arange(grid_width)
    y = np.arange(grid_height)
    grid_points = np.array([[i, j] for i in x for j in y])  # All (x, y) pairs
    n = len(grid_points)
    
    # Compute the covariance matrix
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i, j] = rbf_kernel_2D(grid_points[i], grid_points[j], length_scale, variance)
    
    # Add a small value to the diagonal for numerical stability
    K += 1e-6 * np.eye(n)
    
    # Sample from the multivariate Gaussian distribution
    gp_values = np.random.multivariate_normal(mean * np.ones(n), K)
    
    # Reshape to a 2D grid
    gp_grid = gp_values.reshape((grid_width, grid_height))
    
    return gp_grid

def periodic_kernel(x1, x2, length_scale=1.0, variance=1.0, period=1.0):
    """Periodic kernel function for 1D inputs."""
    dist = np.abs(x1 - x2)
    sin_term = np.sin(np.pi * dist / period)  # Sinusoidal periodicity
    return variance * np.exp(-2 * (sin_term**2) / length_scale**2)

def generate_1d_gaussian_process(extra_points = 0, length_scale=1.0, variance=1.0, period=1.0, inp_arr = [], mean = 0.0, force_mean = True):
    """Generate a 1D Gaussian Process with a periodic kernel."""
    x = inp_arr.copy()
    # Define the 1D grid
    if extra_points > 1:
        x.extend(list(np.linspace(0, 1, extra_points, endpoint=False)))  # Inputs evenly spaced between 0 and 1
    x = np.array(x)
    n = len(x)

    assert n > 1

    # Compute the covariance matrix
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i, j] = periodic_kernel(x[i], x[j], length_scale, variance, period)
    
    # Add a small jitter for numerical stability
    K += 1e-6 * np.eye(n)

    # Sample from the multivariate Gaussian
    gp_samples = np.random.multivariate_normal(mean * np.ones(n), K)

    if force_mean: gp_samples = gp_samples - np.mean(gp_samples) + mean

    assert len(gp_samples) == n
    
    return x, gp_samples

def rsm_probability(y, theta, tau):
    """
    Calculates the probability of a given class label in the model.

    Args:
    y (int): The class label for which the probability is calculated.
    theta (np.ndarray): An array of model parameters.
    tau (np.ndarry): The threshold parameters for the model.

    Returns:
    float: The probability of the given class label.

    """
    unsummed = np.concatenate(([0], theta - tau))
    probs = softmax(np.cumsum(unsummed))
    return probs[y]

def rsm_probability_vector(theta, tau):
    """
    Calculates the probability of set of class labels in given model

    Args:
    theta (np.ndarray): An array of model parameters.
    tau (np.ndarry): The threshold parameters for the model.

    Returns:
    np.ndarray: Array probability of given class label.
    """
    unsummed = np.concatenate(([0], theta - tau))
    probs = softmax(np.cumsum(unsummed))
    return probs

def plot_rater_characteristic_curve(
        taus,
        min_theta=-6,
        max_theta=6,
        resolution=500,
        colors=px.colors.diverging.Spectral,
        dimensions=None,
    ) -> go.Figure:
        """
        Plot the characteristic curves for raters based on the fitted Stan model.

        Args:
            rater_id (int, optional): The rater ID to plot. If None, all raters
                will be plotted. Defaults to None.
            dimensions (tuple, optional): Dimensions of the plot as (width, height).

        Returns:
            A Plotly figure object containing the plotted characteristic curves.
        """
        taus_with_bounds = np.concatenate(([min_theta], taus, [max_theta]))
        x = np.linspace(min_theta, max_theta, int((max_theta - min_theta) * resolution))
        num_categories = len(taus) + 1
        fig = go.Figure()
        for i in range(num_categories):
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=[rsm_probability(i, theta, taus) for theta in x],
                    line=dict(width=2, color=colors[i]),
                    name=str(i + 1),
                )
            )
            fig.add_shape(
                type="rect",
                x0=taus_with_bounds[i],
                x1=taus_with_bounds[i + 1],
                y0=1.02,
                y1=1.1,
                fillcolor=colors[i],
            )
            if i != num_categories - 1:
                fig.add_shape(
                    type="line",
                    x0=taus[i],
                    x1=taus[i],
                    y0=0,
                    y1=1,
                    line=dict(color=colors[i], dash="dot"),
                )

        fig.update_layout(
            xaxis_title="Turf Quality on Latent Scale",
            yaxis_title="Probability",
            legend=dict(x=1.02, y=1),
        )
        if dimensions:
            fig.update_layout(width=dimensions[0], height=dimensions[1])
        return fig

### Parameters for Parameter Recovery

In [ ]:
# Adjustable parameters

# base grid
grid_size_x = 21
grid_size_y = 21

# spatial effect Gaussian Process
lengthscale_plot = 3
sigma_plot = 2

# rating event generation
num_raters = 5
min_rating_events_per_rater = 5
num_rating_events = num_raters * (min_rating_events_per_rater + 5)

# seasonality Gaussian Process
lengthscale_f = 2
sigma_f = 2.0
intercept_f_std = 2.0

# rater beta parameters
num_categories = 9
min_threshold_spacing = 0.3
max_threshold_spacing = 1.2

# Sampling config
config = {'data_path': 'data/raw/quality_nj2.csv',
 'stan_file': 'models/nteprsm_turf_annual_seasonality.stan',
 'stan_additional_data': {'M_f': 8, 'pred_N': 100, 'padding': 5},
 'sampling': {'parallel_chains': 4,
  'seed': 1,
  'show_progress': True,
  'adapt_delta': 0.99,
  'max_treedepth': 15,
  'refresh': 20,
  'iter_warmup': 500,
  'iter_sampling': 1500,
  'save_warmup': True,
  'output_dir': 'data/model_output',
  'time_fmt': '%Y%m%d',
  'show_console': True}}

# filename to save the posterior samples under
savename = 'param_recover_example'

In [ ]:
## generated basic parameters
assert (grid_size_x * grid_size_y) % 3 == 0
num_entries = grid_size_x * grid_size_y // 3

plot_coordinates = [(x, y) for x in range(grid_size_x) for y in range(grid_size_y)]
random.shuffle(plot_coordinates)

# filepath for synthetic dataset
filepath = ROOT_DIR/f"{savename}.csv"

### Synthetic Spatial Effect

In [ ]:
gp_grid = generate_plot_effect_gaussian_process(grid_size_x, grid_size_y, lengthscale_plot, sigma_plot**2)
gp_grid = gp_grid - np.mean(gp_grid)

# Visualization
plt.imshow(gp_grid, cmap="viridis", extent=(0, grid_size_x, 0, grid_size_y))
plt.colorbar(label="GP Value")
plt.title("Plot effect GP")
plt.show()

### Synthetic Rating Times

In [ ]:
rating_event_times = sorted([random.random() for i in range(num_rating_events)])
## TODO : Adjust from march to november (?)

raters = [
    rater
    for rater in range(num_raters)
    for _ in range(min_rating_events_per_rater)
]
remaining_events = num_rating_events - (min_rating_events_per_rater * num_raters)
raters += [random.randint(0, num_raters - 1) for _ in range(remaining_events)]
random.shuffle(raters)

In [ ]:
# Create the figure
plt.figure(figsize=(10, 5))

# Define distinct colors for each rater
colors = plt.cm.get_cmap("Set2", num_raters).colors  # Get unique colors for raters

# Plot each rater separately for better legend control
for rater_id in range(num_raters):
    indices = [i for i, r in enumerate(raters) if r == rater_id]
    plt.scatter(
        np.array(rating_event_times)[indices],  # Use existing event times
        [raters[i] for i in indices], 
        color=colors[rater_id], 
        label=f"Rater {rater_id}", 
        edgecolors="k", 
        alpha=1.0,
        s = 50
    )

# Labels and title
plt.xlabel("Rating Event Time (Normalized)")  # Adjust based on your time format
plt.ylabel("Rater ID")
plt.title("Synthetic Rating Events and Assigned Raters")
plt.yticks(range(num_raters))  # Ensure y-axis only shows valid rater IDs
plt.legend(title="Raters", bbox_to_anchor=(1.05, 1), loc="upper left")  # Move legend outside
plt.grid()

# Improve layout
plt.tight_layout()

# Show plot
plt.show()

### Synthetic Seasonality

In [ ]:
# Params for illustration
illustrated_entries = 5
num_points = 0    # Extra points for illustration
period = 1.0        # Period of the periodic kernel

# gp_samples
gp_samples = []
print("Generating Gaussian Processes...")
print("|"*illustrated_entries)
for i in range(illustrated_entries):
    intercept_f = norm.rvs(loc=0, scale=intercept_f_std)
    x, gp = generate_1d_gaussian_process(num_points, lengthscale_f, sigma_f**2, period, rating_event_times, mean = intercept_f, force_mean = True)
    gp_samples.append(gp)
    print("|", end = '')

# Plot the Gaussian Process
x_sorted = sorted(x)
for i in range(illustrated_entries):
    gp_samples_sorted = [z for _, z in sorted(zip(x, gp_samples[i]))]
    plt.plot(x_sorted, gp_samples_sorted, label=f"Entry {i}")

plt.title("Turfgrass Seasonality GP Samples")
plt.xlabel("Time of Year (Normalized)")
plt.ylabel("Synthetic Seasonality")
plt.grid(True)
plt.legend()
plt.show()

print("Generating the rest of the Gaussian Processes...")
print("|"*(num_entries - illustrated_entries))
for i in range(num_entries - illustrated_entries):
    intercept_f = norm.rvs(loc = 0, scale = intercept_f_std)
    x, gp = generate_1d_gaussian_process(num_points, lengthscale_f, sigma_f**2, period, rating_event_times, mean = intercept_f)
    gp_samples.append(gp)
    print("|", end = '')

### Synthetic Rater Thresholds

In [ ]:
taus = []

for i in range(num_raters):
    ## we want some reasonable looking thresholds
    diffs = [0 for _ in range(num_categories - 2)]
    while min(diffs) < min_threshold_spacing or max(diffs) > max_threshold_spacing:
        thresholds = sorted([norm.rvs(loc=0, scale=2) for _ in range(num_categories - 1)])
        diffs = [thresholds[i+1] - thresholds[i] for i in range(num_categories - 2)]
    print(f"Thresholds for rater {i}: {thresholds}")
    plot_rater_characteristic_curve(thresholds).show()
    taus.append(thresholds)

### Synthetic Dataset Generation

In [ ]:
data = []

for r in range(num_rating_events):
    rating_time = rating_event_times[r]
    rater = raters[r]
    for plot_id in range(len(plot_coordinates)):
        entry_code = plot_id // 3 + 1
        plot_row = plot_coordinates[plot_id][0] + 1
        plot_col = plot_coordinates[plot_id][1] + 1
        plot_effect = gp_grid[plot_row - 1][plot_col - 1]
        turfgrass_seasonality = gp_samples[entry_code - 1][r]
        theta = plot_effect + turfgrass_seasonality
        probs = rsm_probability_vector(theta, taus[rater])
        rating = np.random.choice(len(probs), size=1, p=probs)[0]
        data.append({
            "entry_name": entry_code,
            "plt_id":plot_id,
            "row": plot_row,
            "col": plot_col,
            "rater": rater,
            "rating_event":str(r),
            "adj_time_of_year": rating_time,
            "quality": rating + 1,
            ### for comparing outcomes
            "SEASONALITY": turfgrass_seasonality,
            "PLOT_EFFECT": plot_effect,
            "OUTCOME_PROBABILITY": probs[rating],
        })
        
df = pd.DataFrame(data)
df = df.assign(
    entry_name_code=pd.Categorical(df["entry_name"]).codes,
    plt_id_code=pd.Categorical(df["plt_id"]).codes,
    rater_code=pd.Categorical(df["rater"]).codes,
    rating_event_code=pd.Categorical(df["rating_event"]).codes,
)
df["entry_cumcount"] = df.groupby("entry_name").cumcount() + 1
df.to_csv(filepath)
df.head()

### Posterior Sample Extraction

In [ ]:
datahandler = utils.DataHandler(filepath=filepath)
datahandler.model_data = df
datahandler.generate_stan_data(**config["stan_additional_data"])

In [ ]:
!install_cmdstan

In [ ]:
nteprsm = CmdStanModel(
    stan_file=config["stan_file"],
    stanc_options={"include-paths": get_include()},
)

# samples will be saved in the csv files in the output directory specified in the config
fit = nteprsm.sample(data=datahandler.stan_data, **config["sampling"])

### Saving Posterior Samples

In [ ]:
# Dump the fit object
with open(f"fit_{save_name}.pkl", "wb") as f:
    pickle.dump(fit, f)

# Dump the df object
with open(f'df_{save_name}.pkl', "wb") as f:
    pickle.dump(df, f)

with open(f'gpgrid_{save_name}.pkl', "wb") as f:
    pickle.dump(gp_grid, f)

with open(f'taus_{save_name}.pkl', "wb") as f:
    pickle.dump(taus, f)